In [ ]:
import os
import requests
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

Importando os dados

In [ ]:
diretorio_principal = os.getcwd()
subdiretorio_dados = os.path.join(diretorio_principal, "Dados")

# Criar o diretório se não existir
os.makedirs(subdiretorio_dados, exist_ok=True)

In [ ]:
anos = range(2008, 2024)
formato = "csv"
url_base = "http://www.camara.leg.br/cotas/"

for ano in anos:
    download = requests.get(url_base + f"Ano-{ano}.{formato}.zip")
    caminho_zip = os.path.join(subdiretorio_dados, f"Ano-{ano}.{formato}.zip")

    with open(caminho_zip, "wb") as file:
        file.write(download.content)

    # Extrair o conteúdo do arquivo ZIP
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(subdiretorio_dados)

    # # Remover o arquivo ZIP após extração (opcional)
    # os.remove(caminho_zip)

In [ ]:
# Voltar ao diretório principal
os.chdir(diretorio_principal)

# Criar um DataFrame com os dados extraídos
despesas_totais = []

for arquivo_csv in os.listdir(subdiretorio_dados):
    if arquivo_csv.endswith('.csv') and not arquivo_csv.startswith("."):
        caminho_csv = os.path.join(subdiretorio_dados, arquivo_csv)
        despesas = pd.read_csv(caminho_csv, sep=";", encoding='UTF-8', low_memory=False)
        despesas_totais.append(despesas)

# Concatenar todos os DataFrames em um único DataFrame
despesas_completo = pd.concat(despesas_totais, ignore_index=True)

In [ ]:
despesas_completo = pd.read_csv("Dados/Ano-2008.csv", sep=";", encoding='UTF-8')

In [ ]:
dataset = despesas_completo.loc[:,['sgUF', 'sgPartido', 'txtDescricao', 'txtPassageiro', 'txtTrecho', 'datPagamentoRestituicao', 'vlrRestituicao']]
dataset['datPagamentoRestituicao'] = pd.to_datetime(dataset['datPagamentoRestituicao'], errors='coerce')
dataset = dataset.drop_duplicates(keep="first")
dataset = dataset.dropna()
dataset.head(10)

In [ ]:
despesas_completo

Selecionando Colunas de Interesse

In [ ]:
dataset = despesas_completo.loc[:,['sgUF', 'sgPartido', 'txtDescricao', 'txtPassageiro', 'txtTrecho', 'datPagamentoRestituicao', 'vlrRestituicao']]

In [ ]:
dataset

Conferindo dados Selecionados

In [ ]:
dataset.head(10)

In [ ]:
dataset.tail(10)

In [ ]:
dataset.shape

In [ ]:
dataset.describe()

In [ ]:
dataset.info()

Alterar tipos de dados

In [ ]:
dataset['datPagamentoRestituicao'] = pd.to_datetime(dataset['datPagamentoRestituicao'], errors='coerce')

Limpeza de dados

In [ ]:
#Remover duplicatas
dataset = dataset.drop_duplicates(keep="first")

In [ ]:
#Remover valores
#dataset = dataset.loc[dataset['coluna'] == 'valor']

In [ ]:
#Remover colunas desnecessárias
#dataset = dataset.drop(columns=['coluna1', 'coluna2'])

Dados faltantes

In [ ]:
#Deletar linhas com dados faltantes
dataset = dataset.dropna()

In [ ]:
#Imputar valores
""" imputer = SimpleImputer(minssing_valeues = np.nan, strategy='mean')
imputer.fit(dataset[:,:])
imputer.transform(dataset[:, :]) """

In [ ]:
#Substituir valores
# dataset.replace(old_value, new_value, inplace=True)

Outliers

In [ ]:
# Diagrama de Tukey (boxplot)
gridsize_boxplot = (1, 2)

fig_boxplot = plt.figure(figsize=(12, 8))

boxplot1 = plt.subplot2grid(gridsize_boxplot, (0, 0))
boxplot2 = plt.subplot2grid(gridsize_boxplot, (0, 1))

boxplot1.set_title('Title 1', fontsize=8)
boxplot1.boxplot(dataset['col1'])

boxplot2.set_title('Title 2', fontsize=8)
boxplot2.boxplot(dataset['col2'])

In [ ]:
# Gráfico de dispersão
gridsize_scatter = (1, 2)

fig_scatter = plt.figure(figsize=(12, 8))

scatter1 = plt.subplot2grid(gridsize_scatter, (0, 0))
scatter2 = plt.subplot2grid(gridsize_scatter, (0, 1))

scatter1.set_title('Title 1', fontsize=8)
scatter1.scatter(dataset['col1'])

scatter2.set_title('Title 2', fontsize=8)
scatter2.scatter(dataset['col2'])

In [ ]:
# Histograma
gridsize_hist = (1, 2)

fig_hist = plt.figure(figsize=(12, 8))

hist1 = plt.subplot2grid(gridsize_hist, (0, 0))
hist2 = plt.subplot2grid(gridsize_hist, (0, 1))

hist1.set_title('Title 1', fontsize=8)
hist1.hist(dataset['col1'])

hist2.set_title('Title 2', fontsize=8)
hist2.hist(dataset['col2'])

In [ ]:
# Remover outliers - Winsorization

cols = list(dataset.columns)
cols.remove('col')

winsorization_flooring = 0.05
winsorization_capping = 1 - winsorization_flooring

not_selected_data_flooring = dataset[cols] <= dataset[cols].quantile(winsorization_flooring)
not_selected_data_capping = dataset[cols] >= dataset[cols].quantile(winsorization_capping)

not_selected_data_all = (not_selected_data_flooring & not_selected_data_capping)

data_index = dataset[cols][not_selected_data_all].index
dataset[cols].drop(data_index, inplace=True

Colinearidade

In [ ]:
# Scatterplots
sns.pairplot(X)

In [ ]:
# Pearson correlation coefficient
# The range for a correlation coefficient is between -1 and 1
# Typically, we say that two variables are correlated if 
# their correlation coefficient is greater than 0.5, or less than -0.5.
corr = X.corr()
print("Correlation Coefficients")
print(corr)

Por txtDescricao = PASSAGEM AÉREA 